# 基于U-Net的肺部CT图像病变检测


## 1. 项目背景介绍

肺部疾病，特别是肺癌，已成为全球范围内导致死亡的主要原因之一。早期检测和准确诊断对提高患者的生存率至关重要。传统的医学影像分析依赖于放射科医生的经验，但随着影像数据量的增加，这种方法需要耗费医生大量的时间。因此，自动化的肺部CT图像分析技术应运而生，以协助医生进行更高效和准确的诊断。

在众多的图像分析技术中，深度学习，尤其是卷积神经网络（CNN），已经展示了其在医学影像处理领域的强大能力。U-Net作为一种经典的深度学习网络，因其在图像分割和病变检测任务中的出色表现，受到广泛关注。U-Net的结构特点使其在处理医学影像中的小样本问题时依然能够取得优异的性能，特别是在病变区域的精准定位和分割方面具有显著优势。

本项目旨在利用U-Net深度神经网络对肺部CT图像进行病变检测。通过系统化的训练和优化模型，提升病变检测的准确率和召回率，并评估模型在实际应用场景中的性能。

- 数据集介绍

  COVID-19 Radiography Dataset 数据集包含21165张四种病变的肺部CT图像以及对应图像的肺部区域mask文件。四种病变类别如下：
  - Normal：正常
  - COVID：新型冠状病毒肺炎
  - Viral Pneumonia：病毒性肺炎
  - Lung Opacity：其他类型肺部感染
  
<div class='insertContainerBox'>
  <div class='row'>
    <div class='insertItem' align="center">
      <img src="./others/Normal-10.png" width="200px"/>
      <div>Normal</div>
    </div>
  </div>
  <div class='row'>
    <div class='insertItem' align="center">
      <img src="./others/COVID-11.png" width="200px"/>
      <div>COVID-19</div>
    </div>
  </div>
  <div class='row'>
    <div class='insertItem' align="center">
      <img src="./others/Viral Pneumonia-21.png" width="200px"/>
      <div>Viral Pneumonia</div>
    </div>
  </div>
  <div class='row'>
    <div class='insertItem' align="center">
      <img src="./others/Lung_Opacity-16.png" width="200px"/>
      <div>Lung Opacity</div>
    </div>
  </div>
</div>

- 数据集的整体特征统计分析

 配置好```./src/myconfig/_init_.py ```文件中的数据集路径```data_root_path```，然后运行``` ./src/data/analysis.py ```文件可以得出各病变类型图像的统计分析：

| 病变类型名称   | 数据类型 | 图像大小 | 像素数值范围 | 样本数 | 样本比例 |
| :----------- | :------- | :----- | :----- | :----- | :----- |
| Normal | 灰值图像 | 299*299     |  0-255  | 10192 | 48.15% |
| COVID | 灰值图像 | 299*299     |  0-255  | 3616 | 17.08% |
| Viral Pneumonia | 灰值图像 | 299*299     |  0-255  | 1345 | 6.35% |
| Lung Opacity | 灰值图像 | 299*299     |  0-255  | 6012 | 28.41% |
<div class='insertContainerBox column'>
  <div class='insertItem' style="text-align: center;">
    <img src="./others/analysis.png" width="800px" style="margin: 0 auto; display: block;"/>
  </div>
</div>
由上述图像统计数据可以看出，Normal类图像的样本数量远高于其他类别的图像，Viral Pneumonia类图像的样本数量也远小于其他类别的图像，并且还发现在Viral Pneumonia类样本中，有少量是RGB三通道图像。此外，我们还能发现，在肺部病变检测任务中只有图像的肺部区域具有任务所需要的信息，并且该数据集的mask大小均为256\*256，所以我们可以对图像适当裁剪。因此接下来需要针对这三个问题对数据集预处理。



## 2. 数据预处理

根据上述的数据分析结果，我们需要对数据集执行下述预处理：

*   灰度值转换
*   有效肺部区域裁剪
*   数据增强

数据集预处理的代码在```./src/data/handle.py```文件中，该代码将读取原数据集，并依次执行灰度值转换和有效肺部区域裁剪预处理，然后将处理后的数据集保存在预设的路径下。数据增强代码则是在```./src/data/dataset.py```文件中，使用**tensorflow**框架构造数据集后，在读取数据时动态加载的过程中执行数据增强处理。



下面将逐一介绍预处理的细节。


### 2.1. 灰度值转换
导入```PTL.Image```图像处理库，通过示例代码实现RGB图像转换为灰值图像，运行前注意将图片路径替换为正确的路径。

In [1]:
from PIL import Image

image_path="Your image path"

with Image.open(image_path) as img:
        if img.mode == 'RGB':
            img = img.convert('L')

FileNotFoundError: [Errno 2] No such file or directory: 'Your image path'

### 2.2. 有效肺部区域裁剪
根据原数据集提供的mask文件可以知道，虽然样本图像的大小有299\*299，但我们可以将其裁剪为256\*256大小，能有效剔除图像中部分对本项目无帮助的数据，使模型训练和推理速度提升。



导入```PIL.Image```图像处理库，通过示例代码展示如何实现对有效肺部区域进行裁剪，运行前注意将图片路径替换为正确的路径。

In [ ]:
from PIL import Image

image_path="Your image path"

with Image.open(image_path) as img:
        if img.size == (299, 299):
            left = int((299 - 256) / 2)
            top = 0
            right = int((299 + 256) / 2)
            bottom = 256
            img = img.crop((left, top, right, bottom))


这里不使用mask直接对有效区域和无效区域进行区分的原因是，虽然这能大大提高样本的有效信息比例，但实际生产中的肺部CT图像并没有标注好的mask文件，这会导致训练数据与实际生产中数据的分布不一致，影响方法效果。要解决这个问题我提出以下方法。
- 先训练一个语义分割模型来为未知数据标注mask
- 然后再使用预测的mask与原样本image对分类模型进行训练



这种方法要求分割模型的精度需要非常高，可能需要花费较长的调试时间，因此考虑到本项目所能使用的GPU时间等因素的限制，该方法在本项目中并未尝试。

### 2.3. 数据增强
为了解决样本比例不均衡带来的问题，可以通过在数据动态读取时执行数据增强方法解决。主要使用的数据增强方法有：**随机旋转**和**随机翻转**。

下面是数据增强的示例代码，详细代码可参考```./src/data/dataset.py```文件。

In [ ]:
def augment_image(image, mask):
    if random.random() > 0.5:
        image = tf.image.random_flip_left_right(image)
        mask = tf.image.random_flip_left_right(mask)
    if random.random() > 0.5:
        image = tf.image.random_flip_up_down(image)
        mask = tf.image.random_flip_up_down(mask)

    return image, mask

## 3. 方法概述

本项目选择在图像分割和病变检测任务中表现出色的U-Net模型对肺部CT进行病变检测分类。模型的编码器采用经典的ResNet50作为模型的backbone，并且backbone使用预训练权重，解码器的末端输出使用全局平均池化后接一个全连接层与softmax作为模型的分类头。


模型的完整代码构建代码在```./src/models/resunet.py```文件中，可以运行下面代码查看模型的结构信息。

In [ ]:
import tensorflow as tf
import tensorflow.keras as K
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.applications import ResNet50


def conv_block(inputs, filters, kernel_size=3, activation='relu', padding='same'):
    x = layers.Conv2D(filters, kernel_size, padding=padding, kernel_initializer=K.initializers.he_normal, kernel_regularizer=K.regularizers.l2(5e-4))(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activation)(x)
    return x


def residual_block(inputs, filters, kernel_size=3, activation='relu', padding='same'):
    x = conv_block(inputs, filters, kernel_size, activation, padding)
    x = conv_block(x, filters, kernel_size, activation, padding)
    shortcut = layers.Conv2D(filters, kernel_size=1, padding=padding, kernel_initializer=K.initializers.he_normal, kernel_regularizer=K.regularizers.l2(5e-4))(inputs)
    shortcut = layers.BatchNormalization()(shortcut)
    x = layers.add([x, shortcut])
    x = layers.Activation(activation)(x)
    return x


def upsample_block(inputs, filters, kernel_size=2, strides=2, padding='same'):
    x = layers.Conv2DTranspose(filters, kernel_size, strides=strides, padding=padding, kernel_initializer=K.initializers.he_normal, kernel_regularizer=K.regularizers.l2(5e-4))(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    return x


def ResUNet(input_shape, num_classes):
    input = Input(input_shape)

    inputs = input
    if input_shape[2] == 1:
        inputs = layers.Concatenate()([input, input, input])

    base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=inputs)

    e1 = base_model.get_layer("conv1_relu").output
    e2 = base_model.get_layer("conv2_block3_out").output
    e3 = base_model.get_layer("conv3_block4_out").output
    e4 = base_model.get_layer("conv4_block6_out").output
    e5 = base_model.get_layer("conv5_block3_out").output

    d1 = upsample_block(e5, 512)
    d1 = layers.concatenate([d1, e4])
    d1 = residual_block(d1, 512)

    d2 = upsample_block(d1, 256)
    d2 = layers.concatenate([d2, e3])
    d2 = residual_block(d2, 256)

    d3 = upsample_block(d2, 128)
    d3 = layers.concatenate([d3, e2])
    d3 = residual_block(d3, 128)

    d4 = upsample_block(d3, 64)
    d4 = layers.concatenate([d4, e1])
    d4 = residual_block(d4, 64)

    x = layers.GlobalAveragePooling2D()(d4)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = Model(input, outputs)
    return model

input_shape = (256, 256, 1)
num_classes = 4
model = ResUNet(input_shape, num_classes)
model.summary()

ResUNet模型的结构如下图所示。图像输入到模型后，编码器对其进行多次的卷积特征提取与下采样，生成多份不同尺度的特征张量，然后解码器将这些特征张量融合后输出分类的类别。


<div class='insertContainerBox column'>
  <div class='insertItem' style="text-align: center;">
    <img src="./others/model.png" width="800px" style="margin: 0 auto; display: block;"/>
  </div>
</div>


## 4. 实验与分析

### 4.1 实验配置
首先，编写好myconfig配置类，实验具体配置在代码```./src/myconfig/_init_.py```文件中，下面仅展示配置示例代码。

In [11]:
class MyConfig():
    def __init__(self):
        self.config_name = "p1_resunet" # 配置命名，方便后续模型保存与测试

        self.data_root_path = "/home/jovyan/work/datasets/66836d55111e5909c8b26658-662777f4b3377299d4034f74/COVID-19_Pre_Dataset" # 数据集路径
        self.categories = ['COVID', 'Lung_Opacity', 'Normal', 'Viral Pneumonia'] # 分类类别名称
        self.data_batch_size = 16 # 批次大小

        self.input_shape = (256, 256, 1) # 模型的输入大小
        self.class_num = 4 # 分类数量
 
        self.epoch = 10  # 训练轮次
        self.times_val = 1 # 每隔多少轮验证一次
        self.learn_rate = 1e-5 # 学习率

        self.base_save_path = r"/home/jovyan/work/results/" # 训练结果保存路径

myconfig = MyConfig()
print("configuration")

configuration is complete.


除了上述配置外，本实验还默认使用 **Adam** 作为模型的梯度下降优化器，损失函数默认设置为 **交叉熵损失(cross-entropy)**。

In [14]:
model = ResUNet((256, 256, 1), 4)
model.compile(loss=K.losses.categorical_crossentropy,
            optimizer=K.optimizers.Adam(learning_rate=1e-5))
print("model compile")

model compile


### 4.2 数据集构造
使用 **tensorflow** 框架动态读取数据集，在数据集划分为训练集，验证集和测试集，划分比例为 **8:1:1**。在加载数据时进行归一化，数据增强处理。完整的数据集构造代码在```./src/data/dataset.py```文件中，下面展示示例代码。

In [ ]:
import os
import glob
import random
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

def load_and_preprocess_image(image_path, mask_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.rgb_to_grayscale(image)
    image = tf.image.resize(image, [256, 256])
    image = tf.cast(image, tf.float32) / 255.0

    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=1)
    mask = tf.image.resize(mask, [256, 256])
    mask = tf.cast(mask, tf.float32) / 255.0
    return image, mask


def augment_image(image, mask):
    if random.random() > 0.5:
        image = tf.image.random_flip_left_right(image)
        mask = tf.image.random_flip_left_right(mask)
    if random.random() > 0.5:
        image = tf.image.random_flip_up_down(image)
        mask = tf.image.random_flip_up_down(mask)

    return image, mask


def load_image_labels(dataset_path, categories):
    image_paths = []
    mask_paths = []
    labels = []

    for i, category in enumerate(categories):
        category_images_path = os.path.join(dataset_path, category, 'images', '*.png')
        category_masks_path = os.path.join(dataset_path, category, 'masks', '*.png')

        images = sorted(glob.glob(category_images_path))
        masks = sorted(glob.glob(category_masks_path))

        combined = list(zip(images, masks, [i] * len(images)))
        random.shuffle(combined)
        images, masks, label_list = zip(*combined)

        image_paths.extend(images)
        mask_paths.extend(masks)
        labels.extend(label_list)

    return image_paths, mask_paths, labels


def create_dataset(image_paths, mask_paths, labels, batch_size, num_classes, do_augment=False):
    def generator():
        for img_path, mask_path, label in zip(image_paths, mask_paths, labels):
            yield img_path, mask_path, label

    dataset = tf.data.Dataset.from_generator(
        generator,
        output_types=(tf.string, tf.string, tf.int32),
        output_shapes=((), (), ()),
    )

    def load_image(img_path, mask_path, label):
        image, mask = load_and_preprocess_image(img_path, mask_path)
        label = tf.one_hot(label, num_classes)
        if do_augment is True:
            image, mask = augment_image(image, mask)
        return image, label

    dataset = dataset.map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

    dataset = dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

    return dataset

batch_size = 16
class_num = 4
dataset_path = "/home/jovyan/work/datasets/66836d55111e5909c8b26658-662777f4b3377299d4034f74/COVID-19_Pre_Dataset"
categories = ['COVID', 'Lung_Opacity', 'Normal', 'Viral Pneumonia']

image_paths, mask_paths, labels = load_image_labels(dataset_path, categories)

train_img_paths, val_test_img_paths, train_mask_paths, val_test_mask_paths, train_labels, val_test_labels = train_test_split(
    image_paths, mask_paths, labels, test_size=0.2, random_state=42, stratify=labels
)

val_img_paths, test_img_paths, val_mask_paths, test_mask_paths, val_labels, test_labels = train_test_split(
    val_test_img_paths, val_test_mask_paths, val_test_labels, test_size=0.5, random_state=42,
    stratify=val_test_labels
)

train_dataset = create_dataset(train_img_paths, train_mask_paths, train_labels, batch_size, class_num,
                               do_augment=True)
val_dataset = create_dataset(val_img_paths, val_mask_paths, val_labels, batch_size, class_num, do_augment=False)
test_dataset = create_dataset(test_img_paths, test_mask_paths, test_labels, batch_size, class_num, do_augment=False)

print("Train, validation, and test datasets have been created.")

### 4.3 评估指标
本项目使用了四种评估指标对方法效果进行评估，分别是准确率(accuracy)，精确率(precision)，召回率(recall)和F1-score。因为数据集各类别样本数量差别较大，容易出现某一类别的过拟合，因此在本项目中recall指标会比accuracy，precisison更能体现方法效果。


下面是评估指标的代码示例，完整代码请参考```./src/train.py```和```./src/metric/f1_score.py```文件。

In [15]:
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

        
model.compile(metrics=['accuracy',
                   tf.keras.metrics.Precision(name='precision'),
                   tf.keras.metrics.Recall(name='recall'),
                   F1Score(name='f1_score')])

print("metrics complie")


metrics complie


### 4.4 模型训练和评估

在完成训练配置，数据集构建和评估指标后，可以开始对模型进行训练。这部分完整代码在文件 ```./src/train.py``` 中。


首先实现两个回调方法，用于获取训练过程中的训练结果，方便动态保存结果和模型权重，防止训练中断导致的结果丢失。下面展示评估结果保存 **MetricsCallback** 以及模型保存回调 **SaveBestModelCallback** 实现。

In [ ]:
class MetricsCallback(tf.keras.callbacks.Callback):
    def __init__(self, metrics_save_path):
        super().__init__()
        self.metrics_save_path = metrics_save_path
        self.train_metrics = []
        self.val_metrics = []

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.train_metrics.append({k: v for k, v in logs.items() if not k.startswith('val_')})
        self.val_metrics.append({k: v for k, v in logs.items() if k.startswith('val_')})

        train_df = pd.DataFrame(self.train_metrics)
        val_df = pd.DataFrame(self.val_metrics)

        # 保存到 Excel
        train_df.to_excel(os.path.join(self.metrics_save_path, 'train.xlsx'), index_label='epoch')
        val_df.to_excel(os.path.join(self.metrics_save_path, 'val.xlsx'), index_label='epoch')

        # 绘制折线图
        for metric in train_df.columns:
            plt.figure()
            plt.plot(train_df[metric], label=f'Train {metric}')
            if f'val_{metric}' in val_df.columns:
                plt.plot(val_df[f'val_{metric}'], label=f'Validation {metric}')
            plt.xlabel('Epoch')
            plt.ylabel(metric)
            plt.title(f'Train and Validation {metric}')
            plt.legend()
            plt.savefig(os.path.join(self.metrics_save_path, f'{metric}.png'))
            plt.close()


class SaveBestModelCallback(tf.keras.callbacks.Callback):
    def __init__(self, save_path):
        super().__init__()
        self.save_path = save_path
        self.best_val_accuracy = 0

    def on_epoch_end(self, epoch, logs=None):
        current_val_accuracy = logs.get('val_accuracy')
        if current_val_accuracy and current_val_accuracy > self.best_val_accuracy:
            self.best_val_accuracy = current_val_accuracy
            self.model.save(self.save_path)
            print(f'\nEpoch {epoch+1}: val_accuracy improved to {current_val_accuracy:.4f}, saving model to {self.save_path}')


模型训练代码如下所示。***注意该代码在这里无法直接运行，需要导入自定义的 MyConfig 配置包。完整代码请参考 ```./src/train.py``` 文件***

In [ ]:
def creat_model(load_path):
    config = MyConfig()
    if load_path is not None:
        print("model loading")
        model = K.models.load_model(load_path, compile=False)
        print("load finish")
    else:
        model = ResUNet(config.input_shape, config.class_num)
    model.compile(loss=K.losses.categorical_crossentropy,
                  optimizer=K.optimizers.Adam(learning_rate=config.learn_rate),
                  metrics=['accuracy',
                           tf.keras.metrics.Precision(name='precision'),
                           tf.keras.metrics.Recall(name='recall'),
                           F1Score(name='f1_score')])
    return model


def trainer(train_dataset, val_dataset, load_path):
    config = MyConfig()
    model = creat_model(load_path)
    uid = datetime.datetime.now().strftime(f"%Y%m%d_%H%M%S/{config.config_name}")

    epochs = config.epoch
    BATCH_SIZE = config.data_batch_size

    base_save_path = os.path.join(config.base_save_path, uid)
    model_save_path = os.path.join(base_save_path, 'model.h5')
    metrics_save_path = os.path.join(base_save_path, 'metrics')
    os.makedirs(metrics_save_path, exist_ok=True)

    metrics_callback = MetricsCallback(metrics_save_path)
    save_best_model_callback = SaveBestModelCallback(model_save_path)

    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        batch_size=BATCH_SIZE,
        epochs=epochs,
        verbose=1,
        callbacks=[metrics_callback, save_best_model_callback])

模型训练结果如下图所示。（由于 GPU 使用时间有限，实验仅进行了10轮训练，但仍能得到较好的结果）


<div class='insertContainerBox'>
  <div class='row'>
    <div class='insertItem' align=center><img src="./others/loss.png" width="450px"/></div>
    <div class='insertItem' align=center><img src="./others/accuracy.png" width="450px"/></div>
    <div class='insertItem' align=center><img src="./others/recall.png" width="450px"/></div>
  </div>
  <div class='row'>
    <div class='insertItem' align=center><img src="./others/precision.png" width="450px"/></div>
    <div class='insertItem' align=center><img src="./others/f1_score.png" width="450px"/></div>
    <div class='insertItem' align=center></div>
  </div>
</div>

根据 **loss** 的变化曲线可以看出，在训练到第 7 轮的时候，训练集的损失下降了，但验证集的损失上升了，这可能表明模型开始出现过拟合现象。但在后续的几个 epoch，验证集的性能有所恢复。


可以运行下面代码来展示详细的训练结果。

In [3]:
import pandas as pd
# 读取训练数据文件
train_df = pd.read_excel('/home/jovyan/work/results/20240708_105439/p1_resunet/metrics/train.xlsx')
# 读取验证数据文件
val_df = pd.read_excel('/home/jovyan/work/results/20240708_105439/p1_resunet/metrics/val.xlsx')
# 打印训练数据表格
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.float_format', '{:.4f}'.format)
print(train_df)
print("\nValidation Data:")
print(val_df)

Training Data:
   epoch   loss  accuracy  precision  recall  f1_score
0      0 5.6412    0.7046     0.8814  0.5412    0.6706
1      1 4.1724    0.8903     0.9170  0.8454    0.8797
2      2 3.4676    0.9248     0.9387  0.9098    0.9240
3      3 3.0101    0.9431     0.9516  0.9337    0.9426
4      4 2.6628    0.9572     0.9631  0.9514    0.9572
5      5 2.3912    0.9681     0.9732  0.9641    0.9686
6      6 2.1604    0.9766     0.9792  0.9739    0.9765
7      7 1.9676    0.9804     0.9824  0.9781    0.9803
8      8 1.7985    0.9849     0.9867  0.9831    0.9849
9      9 1.6472    0.9868     0.9880  0.9858    0.9869

Validation Data:
   epoch  val_loss  val_accuracy  val_precision  val_recall  val_f1_score
0      0    5.4803        0.4315         0.5760      0.2793        0.3762
1      1    3.6919        0.9414         0.9523      0.9244        0.9381
2      2    3.2188        0.9338         0.9407      0.9220        0.9313
3      3    2.8391        0.9405         0.9453      0.9386       

为验证方法的有效性，还需要使用测试集对方法进行测试。下面展示测试代码，完整测试代码请参考文件 ```./src/train.py``` 。

In [ ]:
def test_model(test_dataset, load_path, save_path):
    model = creat_model(load_path)

    # 预测整个测试集
    results = model.evaluate(test_dataset)
    metrics_names = model.metrics_names
    results_dict = dict(zip(metrics_names, results))

    # 保存整体测试结果
    with open(os.path.join(save_path, 'test_all_result.txt'), 'w') as f:
        for key, value in results_dict.items():
            f.write(f"{key}: {value}\n")

    # 绘制并保存混淆矩阵
    y_true = []
    y_pred = []

    for images, labels in test_dataset:
        y_true.extend(tf.argmax(labels, axis=1).numpy())
        y_pred.extend(tf.argmax(model.predict(images), axis=1).numpy())

    cm = confusion_matrix(y_true, y_pred)
    print(cm)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix for All Classes')
    plt.savefig(os.path.join(save_path, 'confusion_matrix_all.png'))
    plt.close()

    # 按类别评估
    config = MyConfig()
    class_names = config.categories
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    for i, class_name in enumerate(class_names):
        idx = (y_true == i)
        class_y_true = y_true[idx]
        class_y_pred = y_pred[idx]

        # class_report = classification_report(class_y_true, class_y_pred, output_dict=True, zero_division=0)
        class_report = classification_report(class_y_true, class_y_pred, output_dict=True)
        with open(os.path.join(save_path, f'test_{class_name}_result.txt'), 'w') as f:
            for key, value in class_report.items():
                f.write(f"{key}: {value}\n")

在测试集上的测试结果可以通过运行下面代码来查看。

In [1]:
file_path = r"./others/test_all_result.txt"
with open(file_path, 'r', encoding='utf-8') as file:
    content = file.read()
print(content)

loss: 2.253380298614502
accuracy: 0.968351423740387
precision: 0.9696969985961914
recall: 0.967406690120697
f1_score: 0.9685503840446472



通过评估指标可以观察到，在测试集上，所使用方法的整体**准确率达到 96.83%，精确率 96.97%，召回率 96.74，f1-score 96.85%**。有效证明了所使用方法在肺部CT病变检测分类任务中具有较高的准确性和泛用性。为了跟进一步了解各肺部病变类别的分类情况，下面计算出了各病变类别的分类情况混淆矩阵。



<div class='insertContainerBox column'>
    <div class='insertItem' style="text-align: center;"><img src="./others/confusion_matrix_all.png" width="600px" style="margin 0 auto; display: block;"/>
    </div>
</div>



图中各预测类别 0，1，2，3 分别对应病变类别 COVID，Lung Opacity，Normal，Viral Pneumonia。根据混淆矩阵可以得出，所使用模型能有效检测出COVID的召回率为 96.41%，Lung Opacity召回率为94.85%，Normal 召回率为98.23%，Viral Pneumonia 召回率为 96.27%。该结果证明了所使用方法能较好地根据肺部CT图像检测出是否存在病变以及病变的类型，能有效辅助肺部CT影像分析。


但结果也显示出 Lung Opacity 的召回率偏低，Normal的召回率偏高，而且基本上是 Lung Opacity 错判为 Normal，而 Normal 错判为 Lung Opacity。经分析我得出以下两点可能的原因：
- 模型一定程度上受到数据集的类别样本数量分布不均的影响，导致模型训练有一定的过拟合。
- Lung Opacity 类别代指其他类型肺部感染，含盖的肺部病症表现较多，且部分病症表现较轻微，模型难以学习到部分细微特征所导致。


## 5. 总结与思考
本项目通过对COVID-19 Radiography Dataset数据集进行深入分析和预处理，构建并训练了一个基于U-Net架构的肺部CT图像分类模型，最终实现了对四种肺部病变的高效检测与分类。以下是项目的主要工作和成果总结：

- 数据集分析与预处理：

 对数据集进行了详细的特征统计分析，明确了各类样本数量、图像大小、像素值范围等信息。针对样本不均衡和数据分布问题，进行了灰度值转换和有效肺部区域裁剪，提高了图像的有效信息比例。采用数据增强方法来解决样本数量不均衡问题，增强了模型的泛化能力。
- 模型设计与训练：

 选择了在图像分割和病变检测任务中表现优异的U-Net模型，采用ResNet50作为backbone。编写了模型训练、评估以及保存回调函数，确保训练过程中的结果和模型权重能够及时保存，避免因中断导致的数据丢失。训练过程中，模型在训练集和验证集上的性能均表现出色，尽管在后期出现了轻微的过拟合现象，但整体效果仍然较好。
模型评估：

 使用准确率、精确率、召回率和F1-score四种评估指标对模型效果进行了全面评估。模型在测试集上达到了较好的分类效果，证明了模型在肺部CT病变检测分类任务中的高准确性和泛用性。
 
尽管本项目所使用的方法取得了较好的效果，但在以下几个方面仍存在改进空间：

- 数据不均衡问题：

 未来可以尝试使用带权重的交叉熵损失函数，更复杂的数据增强方法，或通过生成对抗网络（GAN）生成更多的Lung Opacity类样本，以平衡数据分布。

- 细微特征学习：

 Lung Opacity类别涵盖了多种肺部感染，且部分病症表现较轻微，模型难以学习到这些细微特征。未来可以尝试使用更深层次的卷积神经网络或使用注意力机制的方法，以增强模型对细微特征的学习能力。

- 模型优化与调优：

 尽管ResNet50作为backbone取得了较好的效果，但仍有可能通过尝试其他更先进的网络结构（如Vision Transformer）来进一步提升模型性能。进一步优化训练超参数（如学习率、批量大小等），以及采用更长时间的训练，可以帮助模型达到更好的性能。
实际应用场景适应：

 由于实际生产环境中的肺部CT图像没有标注好的mask文件，本项目提出了先训练一个语义分割模型来为未知数据标注mask的方法。尽管该方法在本项目中未尝试，但在实际应用中具有很大的潜力。未来可以考虑实施这一方法，并评估其实际效果。


基于上述思考，未来的工作可以集中在以下几个方面：

- 进一步数据增强和生成：

 探索更多的数据增强技术以及生成对抗网络（GAN）等生成模型，以进一步平衡各类别样本数量，提高模型在小样本类别上的表现。

- 模型架构创新：

 尝试引入更先进的神经网络架构，如Vision Transformer等，以提升模型的整体性能和对细微特征的捕捉能力。

- 多任务学习与联合训练：
 
 结合多任务学习的方法，将图像分割与病变检测任务联合起来训练，以提高模型对不同任务的适应性和综合性能。

- 扩展应用：

 将方法推广应用到其他类型的医学影像数据上，如X射线、MRI等，以验证方法的通用性和实际应用效果。
通过这些进一步的研究和改进，可以更好地提升模型在肺部CT图像分类任务中的性能，推动相关技术在实际临床诊断中的应用。